In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import cv2
import os
%matplotlib inline

# Import everything needed to edit/save/watch video clips
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [2]:
#reading in an image
image = mpimg.imread('test_images/solidYellowCurve2.jpg')
image_shape = image.shape

In [33]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def sharpen(img, alpha=30):
    blurred = gaussian_blur(img, 3)
    filter_blurred = gaussian_blur(blurred,1)
    sharpened = blurred + alpha*(blurred-filter_blurred)
    return sharpened
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def line_locker(LHS_avg_slope, LHS_avg_intercept, RHS_avg_slope, RHS_avg_intercept, y_min, y_max):
    global LHS_locker 
    global RHS_locker
    global first_frame
    
    
    LHS = np.array([(LHS_avg_slope, LHS_avg_intercept)], dtype = [('slope',float),('intercept', float)])
    RHS = np.array([(RHS_avg_slope, RHS_avg_intercept)], dtype = [('slope',float),('intercept', float)])
    
    if first_frame:
        LHS_locker = LHS
        RHS_locker = RHS
        global first_frame
        first_frame = False
    else:
        LHS_locker = np.concatenate((LHS_locker, LHS))
        RHS_locker = np.concatenate((RHS_locker, RHS))
        
    global y_min_global
    y_min_global = y_min
    global y_max_global
    y_max_global = y_max
    
    return

def process_lines(lines, testing = False):
    
    """
    Lines in format: [[X-start, Y-start, X-end, Y-end]]
    """
    
    center = image_shape[1]/2
    y_max = image_shape[0]
    
    holder = []
    LHS_lines = []
    RHS_lines = []
    
    for line in lines:
        if (line[0][0] < center) & (line[0][2] < center):
            LHS_lines.append(tuple(line[0]))
        elif (line[0][0] >= center) & (line[0][2] >= center):
            RHS_lines.append (tuple(line[0]))
    
    LHS_lines = np.array(LHS_lines, dtype = [('X1', int), ('Y1',int), ('X2', int), ('Y2', int)])
    RHS_lines = np.array(RHS_lines, dtype = [('X1', int), ('Y1',int), ('X2', int), ('Y2', int)])
    
    
    n=2
    LHS_lines.sort(order = 'Y1')
    LHS_high = LHS_lines[-n:]
    
    LHS_lines.sort(order = 'Y2')
    LHS_low = LHS_lines[0:n]
    
    RHS_lines.sort(order = 'Y2')
    RHS_high = RHS_lines[-n:]
    RHS_lines.sort(order = 'Y1')
    RHS_low = RHS_lines[0:n]
    
    new_LHS_lines = []
    new_RHS_lines = []

    for i in LHS_high:
        for j in LHS_low:
            new_LHS_lines.append(tuple([i[0],i[1],j[2],j[3]]))

    for i in RHS_high:
        for j in RHS_low:
            new_RHS_lines.append(tuple([j[0],j[1],i[2],i[3]]))
    
    new_LHS_lines = np.array(new_LHS_lines, dtype = [('X1', int), ('Y1',int), ('X2', int), ('Y2', int)])
    new_RHS_lines = np.array(new_RHS_lines, dtype = [('X1', int), ('Y1',int), ('X2', int), ('Y2', int)])
            
    new_LHS_slopes = np.array((new_LHS_lines['Y2']-new_LHS_lines['Y1'])/(new_LHS_lines['X2']-new_LHS_lines['X1']))
    new_LHS_intercepts = np.array((new_LHS_lines['Y2']-new_LHS_lines['X2']*new_LHS_slopes))
    LHS_avg_slope = np.median(new_LHS_slopes)
    LHS_avg_intercept = np.median(new_LHS_intercepts)
    
    new_RHS_slopes = np.array((new_RHS_lines['Y2']-new_RHS_lines['Y1'])/(new_RHS_lines['X2']-new_RHS_lines['X1']))
    new_RHS_intercepts = np.array((new_RHS_lines['Y2']-new_RHS_lines['X2']*new_RHS_slopes))
    RHS_avg_slope = np.median(new_RHS_slopes)
    RHS_avg_intercept = np.median(new_RHS_intercepts)
    
    y_min = min(LHS_lines['Y1'].min(), RHS_lines['Y2'].min(), y_min_global)
    y_max = max(LHS_lines['Y2'].max(), RHS_lines['Y1'].max(), y_max_global)
    
    line_locker(LHS_avg_slope, LHS_avg_intercept, RHS_avg_slope, RHS_avg_intercept, y_min, y_max)
    
    global LHS_locker
    global RHS_locker
    
    m = 5
    LHS_avg_slope = LHS_locker['slope'][-m:].mean()
    LHS_avg_intercept = LHS_locker['intercept'][-m:].mean()
    RHS_avg_slope = RHS_locker['slope'][-m:].mean()
    RHS_avg_intercept = RHS_locker['intercept'][-m:].mean()
    
    LHS_final_line = [[int((y_min-LHS_avg_intercept)/LHS_avg_slope), y_min, int((y_max-LHS_avg_intercept)/LHS_avg_slope), y_max]]
    RHS_final_line = [[int((y_max-RHS_avg_intercept)/RHS_avg_slope), y_max, int((y_min-RHS_avg_intercept)/RHS_avg_slope), y_min]]

    smoothed_lines = [LHS_final_line, RHS_final_line]
    
    line_locker(LHS_avg_slope, LHS_avg_intercept, RHS_avg_slope, RHS_avg_intercept, y_min, y_max)
    
    return smoothed_lines


def draw_lines(lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """

    img = np.zeros((image_shape[0], image_shape[1], 3), dtype=np.uint8)
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            
    return img

    
    #print('Avg len of lines: {0}'.format(cum_length/len(lines)))
        
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
    
    Lines in format: [X-start, Y-start, X-end, Y-end]
    
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    #line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    #draw_lines(line_img, lines)
    return lines

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.6, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [34]:
def process_image(image, to_print = False):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
  
    grayed = grayscale(image)
    
    kernel_size = 1
    blurred = gaussian_blur(grayed, kernel_size)


    low_threshold = 75
    high_threshold = low_threshold*3 # 3:1 ration suggested by Canny
    cannied = canny(blurred, low_threshold, high_threshold)
    
    #canny_blurred = gaussian_blur(cannied,1)
    canny_blurred = cannied

    # mask_vertices defines region of interest within image
    # left-bottom, right-bottom, apex
    x_max = image.shape[1]
    y_max = image.shape[0]
    #(BL, UL, UR, BR)
    mask_vertices = np.array([[(0,y_max),(x_max/2-30, y_max/2+60), (x_max/2+30, y_max/2+60), (x_max,y_max)]], dtype=np.int32)
    #mask_vertices = np.array([[(0,540), (960,540), (480,320)]])
    masked = region_of_interest(canny_blurred, mask_vertices)


    rho = 1
    theta = 1
    threshold = 10
    min_line_len = 15
    max_line_gap = 50
    houghed_lines = hough_lines(masked, rho, theta, threshold, min_line_len, max_line_gap)
    
    processed_lines = process_lines(houghed_lines, testing = to_print)
    
    drawn_lines = draw_lines(processed_lines)
    #drawn_lines = draw_lines(houghed_lines)
    
    drawn_lines= gaussian_blur(drawn_lines, 7)
    
    result = weighted_img(drawn_lines, image)
    #result = weighted_img(smoothed, image)
    
    if to_print:
        fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(48,33))

        im1 = ax1.imshow(canny_blurred, cmap='gray')

        im2 = ax2.imshow(drawn_lines)
        #im2 = ax2.imshow(smoothed)

        im3 = ax3.imshow(result)
        #im3 = ax3.imshow(smoothed)

        plt.show()

    return result

In [35]:
# Format for Line Locker: [[slope, intercept]]
first_frame = True
LHS_locker = np.empty([1,],dtype = [('slope', float), ('intercept',float)])
RHS_locker = np.empty([1,],dtype = [('slope', float), ('intercept',float)])
y_min_global = 960
y_max_global = 0

In [36]:
white_output = 'white.mp4'

clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|██████████████████████████████████████████████████████▊| 221/222 [00:17<00:00, 12.77it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

Wall time: 19.3 s


In [37]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [38]:
LHS_locker

array([(-0.73134094,  655.17062703), (-0.73134094,  655.17062703),
       (-0.73134094,  655.17062703), (-0.73134094,  655.17062703),
       (-0.72201537,  653.22465062), (-0.72947583,  654.78143175),
       (-0.72370387,  653.77297176), (-0.72757539,  654.42406164),
       (-0.74393418,  662.93493409), (-0.72934093,  655.82760997),
       (-0.71142754,  647.60255948), (-0.72719638,  654.91242739),
       (-0.70522588,  645.35926223), (-0.72342498,  653.32735863),
       (-0.72454354,  652.23718399), (-0.71836366,  650.68775834),
       (-0.73      ,  653.535     ), (-0.72031161,  651.02931264),
       (-0.72408836,  652.35739832), (-0.72346143,  651.96933066),
       (-0.71661998,  651.68860878), (-0.72289628,  652.11593008),
       (-0.7062136 ,  649.0023327 ), (-0.71865593,  651.42672011),
       (-0.70619211,  649.5626824 ), (-0.71411558,  650.75925481),
       (-0.72853318,  657.92579843), (-0.71474208,  651.73535769),
       (-0.70515565,  646.21278455), (-0.71374772,  651.239175

In [ ]:
LHS_locker = np.array(LHS_locker, dtype = [('slope',float),('intercept', float)])

In [32]:
LHS_locker['slope']

array([-0.73134094])

In [23]:
test = np.array([1,2,3])

In [24]:
test[-5:]

array([1, 2, 3])